In [56]:
import configparser, datetime, glob, os, re, smtplib
from dateutil import parser
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import ACILib

def send_email(shift='apjc',tech_strip='aci',date=datetime.datetime.today().strftime("%Y-%m-%d"),config={}):

    if config[shift]['email_enabled_{}'.format(tech_strip.lower())] == 'no':
        print("Disable emaling to {} {}".format(shift,tech_strip))
        return
    else:
        #pass
        print("Sending email to {} {}".format(shift,tech_strip))
        
    messagefromfile = ''
    messagesofdates = []
    message_for_date = {}
    
    #today_date = datetime.datetime.today().strftime("%Y-%m-%d")
    #today_weekday = datetime.datetime.today().strftime("%A")
    date_weekday = parser.parse(date).strftime("%A")
    
    container_path = '/home/jovyan/'
    reportpath = container_path+config[shift]['_'.join(['reportpath',tech_strip.upper()])]
    htmlpath = '{}/html/'.format(container_path+config[shift]['frontend_html'],)
    
    messagefiles = glob.glob(htmlpath+'../../*_message.txt')
    for messagefile in messagefiles:
        messagefromfile = ''
        with open(messagefile) as f:
            messagefromfile = f.read()
        f.close()
    messagesofdates = messagesofdates + messagefromfile.split("\n")
    for message in messagesofdates:
        if not message:
            continue
        #<p style="color:blue">This is another paragraph.</p>
        message_for_date[message[:10]] = '<br><p style="color:red">{}</p><br>\n'.format(message[11:])
    if date in message_for_date.keys():
        print(message_for_date[date])
    else:
        message_for_date[date] = ''
        
    FromEmail = config[shift]['email_from_{}'.format(tech_strip.lower())]
    
    #ToEmail = [ccoid+'@cisco.com' for ccoid in  config[shift]['workgroup_{}_mgr'.format(tech_strip.lower())].split(',')]
    ToEmail = ['zdazhi@cisco.com']
    #CCEmail = [ccoid+'@cisco.com' for ccoid in  config[shift]['workgroup_{}_tl'.format(tech_strip.lower())].split(',')]
    CCEmail = ['zdazhi@cisco.com','zdazhi@cisco.com']
    BCCEmail = ['zdazhi@cisco.com']
    emailhost = "outbound.cisco.com"

    # Create message container - the correct MIME type is multipart/alternative.
    msg = MIMEMultipart('alternative')
    msg['Subject'] = "{} {}-{} Tracking Report - {} [SAMPLE v0.2]".format(date,shift.upper(),tech_strip.upper(),date_weekday)
    msg['From'] = FromEmail
    msg['To'] = ", ".join(ToEmail)
    msg['Cc'] = ", ".join(CCEmail)
    
    reportfile = "{}{}_{}_{}.html".format(reportpath,date,shift.upper(),tech_strip.upper())
    with open(reportfile,'r') as file:
        dailyreportstringhtml = file.read()
    
    bottom_html = '<br>This is system-generated email.<br>'
    if shift == 'emea' and tech_strip == 'aci':
        bottom_html = bottom_html + 'Live report is also available at <a href="https://syd-htts-prd.cisco.com/{}_EU">{}_EU Report</a>.  '.format(tech_strip.upper(),tech_strip.upper())
    else:
        bottom_html = bottom_html + 'Live report is also available at <a href="https://syd-htts-prd.cisco.com/{}">{} Report</a>.  '.format(tech_strip.upper(),tech_strip.upper())
    bottom_html = bottom_html + 'Conact <a href="mailto:zdazhi@cisco.com">Derek</a> for any issue.'

    #part1 = MIMEText(head_html, 'text')
    part2 = MIMEText(message_for_date[date]+dailyreportstringhtml+bottom_html, 'html')
    
    print("From {} To {} CC {} BCC {}".format(FromEmail,ToEmail,CCEmail,BCCEmail))
    print("Email Body {}".format(part2))
    
    #msg.attach(part1)
    msg.attach(part2)
    mail = smtplib.SMTP(emailhost)
    mail.ehlo()
    mail.starttls()
    #mail.login('userName', 'password')
    #mail.sendmail(FromEmail, ToEmail+CCEmail+BCCEmail, msg.as_string())
    mail.quit()
    

In [58]:
#Reading current config from file
#Docker WORKDIR is /HTTSDashboard/Apps/
config = configparser.ConfigParser()
config.read('./tacconfig.ini')

shift = 'emea'
tech_strip_list = ['sv','aci','dcrs']
tech_strip_list = ['aci']
for tech_strip in tech_strip_list:
    #send_email(shift=shift,tech_strip=tech_strip,date=datetime.datetime.today().strftime("%Y-%m-%d"),config=config)
    send_email(shift=shift,tech_strip=tech_strip,date='2020-07-13',config=config)

From reportbot-aci To ['zdazhi@cisco.com'] CC ['zdazhi@cisco.com', 'zdazhi@cisco.com'] BCC ['zdazhi@cisco.com']
Email Body Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

2020-07-13 Queue Case Volume -- Total 17 *** Update at(IST): 2020-07-13 18:33:02<br>
==================<br>
FTS:3<br>
UC:4<br>
EME-ACI-Solutions:10<br>
<br>
2020-07-13 SR ownership taken by workgroup - 9 cases.<br>
==================<br>
WORKGROUP APT-ACI-SOLUTIONS - 1 engineers 1 cases<br>
Case Taken by Severity P1:0 P2:1 P3:0 P4:0<br>
wilchong 1 ['689452698']<br>
<br>
WORKGROUP GCE-ACI-Solutions - 6 engineers 8 cases<br>
Case Taken by Severity P1:0 P2:2 P3:6 P4:0<br>
kdoodi 1 ['689452698']<br>
deseth2 2 ['689477569', '689478263']<br>
visgupt2 1 ['689467209']<br>
prpratee 1 ['689477931']<br>
bharatkc 1 ['689478151']<br>
birai 2 ['689474894', '689417011']<br>
<br>
2020-07-13 Realtime events - Total Cases(incl. FTS/UC): 17<br>
 Queue: EME-ACI-SOLUTIONS <br>
===============